In this notebook we compare the two methods

    *FTS Stokesian dynamics via a direct solver
    *PyStokes: superposition approximation
    
for two spheres falling in a direction perpendicular to their line of centres due to gravity. We shall look at velocity and angular velocity. Both are assumed to be zero at the start of the fall. 

Here, we want to investigate whether we can match PyStokes and FTS by considering the factors beyond superposition approximation.

<img src="./images/IMG_1346.jpeg" style="width: 500px;"/>

Explicitly, these are 

    *G1s2s * F2s
    *G2a2s * F2s
    
for the velocity and angular velocity, respectively. 

In [1]:
from ftssd import FTS
import autograd.numpy as np, matplotlib.pyplot as plt

<font size="5">FTS Stokesian dynamics

In [2]:
# particle radius, self-propulsion speed, number and fluid viscosity
b, Np, eta = 0.5, 2, 0.1
fts = FTS(b, Np, eta)

In [3]:
%%time
## Two spheres falling next to each other: FTS

#initial position
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -1.,-1.])
T = np.zeros(3*Np)

# integration parameters
Nt=2; r1=np.zeros([3,Nt]); r2=np.zeros([3,Nt])
r1[:,0] = r[::Np]; r2[:,0] = r[1::Np]
dt=3.;
v = np.zeros(3*Np);
o = np.zeros(3*Np)
theta1=np.zeros([3,Nt]); theta2=np.zeros([3,Nt])
v1 = np.zeros([3,Nt-1]); v2 = np.zeros([3,Nt-1])
o1 = np.zeros([3,Nt-1]); o2 = np.zeros([3,Nt-1])

for i in range(Nt-1):
    fts.directSolve(v, o, r, F, T)
    
    r1[:,i+1] = r1[:,i] + dt*v[::Np]
    r2[:,i+1] = r2[:,i] + dt*v[1::Np]
    
    theta1[:,i+1] = theta1[:,i] + dt*o[::Np]
    theta2[:,i+1] = theta2[:,i] + dt*o[1::Np]
    
    #get velocity (constant for this case)
    v1[:,i] = v[::Np]
    v2[:,i] = v[1::Np]
    
    #get angular velocity (constant for this case)
    o1[:,i] = o[::Np]
    o2[:,i] = o[1::Np]
    
    # reset variables for next time step
    r[::Np] = r1[:,i+1]
    r[1::Np] = r2[:,i+1]
    v = v*0
    o = o*0

print('finish')

finish
CPU times: user 4.14 s, sys: 25.2 ms, total: 4.17 s
Wall time: 4.16 s


In [4]:
## velocity and angular velocity of particle 1
v1[:,0], o1[:,0]

(array([ 0.        ,  0.        , -1.24025471]),
 array([0.        , 0.08191567, 0.        ]))

--------------------------------

<font size="5">PyStokes (superposition approximation)

In [5]:
import pystokes

In [6]:
pstk = pystokes.unbounded.Rbm(b, Np, eta)

In [7]:
%%time
## Two spheres falling next to each other: pystokes

#initial position
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -1.,-1.])
T = np.zeros(3*Np)

# integration parameters
Nt=2; r1=np.zeros([3,Nt]); r2=np.zeros([3,Nt])
r1[:,0] = r[::Np]; r2[:,0] = r[1::Np]
dt=3.;
v = np.zeros(3*Np);
o = np.zeros(3*Np)
theta1=np.zeros([3,Nt]); theta2=np.zeros([3,Nt])
v1_pstk = np.zeros([3,Nt-1]); v2_pstk = np.zeros([3,Nt-1])
o1_pstk = np.zeros([3,Nt-1]); o2_pstk = np.zeros([3,Nt-1])

for i in range(Nt-1):
    pstk.mobilityTT(v, r, F)
    pstk.mobilityTR(v, r, T)
    pstk.mobilityRT(o, r, F)
    pstk.mobilityRR(o, r, T)
    
    r1[:,i+1] = r1[:,i] + dt*v[::Np]
    r2[:,i+1] = r2[:,i] + dt*v[1::Np]
    
    theta1[:,i+1] = theta1[:,i] + dt*o[::Np]
    theta2[:,i+1] = theta2[:,i] + dt*o[1::Np]
    
    #get velocity (constant for this case)
    v1_pstk[:,i] = v[::Np]
    v2_pstk[:,i] = v[1::Np]
    
    #get angular velocity (constant for this case)
    o1_pstk[:,i] = o[::Np]
    o2_pstk[:,i] = o[1::Np]
    
    # reset variables for next time step
    r[::Np] = r1[:,i+1]
    r[1::Np] = r2[:,i+1]
    v = v*0
    o = o*0

print('finish')

finish
CPU times: user 784 µs, sys: 702 µs, total: 1.49 ms
Wall time: 944 µs


In [8]:
## velocity and angular velocity of particle 1
v1_pstk[:,0], o1_pstk[:,0]

(array([ 0.        ,  0.        , -1.23947782]),
 array([0.      , 0.075215, 0.      ]))

-----------------------------

Aside: In free space, for two bodies, the first order in the Jacobi iteration is the same as the zeroth order (=superposition approximation), so there will be no correction. Does that seem right?

Corrections appear at three bodies or when there are boundaries.

---------------------------------


<font size="5">Do they match?

Compute the contribution we get from G1s2s F2s

In [9]:
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -1.,-1.])
T = np.zeros(3*Np)

i=0; j=1

xij = r[i]    - r[j]
yij = r[i+Np]  - r[j+Np]
zij = r[i+2*Np]  - r[j+2*Np]

force  = np.array([F[j],F[j+Np], F[j+2*Np]])
torque = np.array([T[j],T[j+Np], T[j+2*Np]])

lhs = fts.tensorG2s2s(xij,yij,zij)
lhs_mat = np.reshape(lhs, (9,9))
lhs_mat_inv = np.linalg.pinv(lhs_mat)
lhs_inv = np.reshape(lhs_mat_inv, (3,3,3,3))
rhs = (np.dot(fts.tensorG2s1s(xij,yij,zij), force) 
                + 1./b * np.dot(fts.tensorG2s2a(xij,yij,zij), torque))
F2s = np.einsum('ijkl, kl', lhs_inv, rhs)


In [10]:
F2s

array([[ 0.        ,  0.        , -0.13659985],
       [ 0.        ,  0.        ,  0.        ],
       [-0.13659985,  0.        ,  0.        ]])

See whether if we add this to v and o from pystokes with G1s2s and G2a2s it becomes the same. 

In [11]:
v_ = - np.einsum('ijk,jk',fts.tensorG1s2s(xij,yij,zij),F2s)

In [12]:
v_

array([-0.        , -0.        , -0.00077689])

In [13]:
## velocity difference between FTS and pystokes
v1 - v1_pstk

array([[ 0.        ],
       [ 0.        ],
       [-0.00077689]])

Indeed, the difference between the velocities of FTS and PyStokes is equal to G1s2s * F2s.

________________________________


Now compare the angular velocity

In [14]:
o_ = - np.einsum('ijk,jk',fts.tensorG2a2s(xij,yij,zij),F2s)
o_

array([-0.        ,  0.00670067, -0.        ])

In [15]:
## angular velocity difference between FTS and pystokes
o1 - o1_pstk

array([[0.        ],
       [0.00670067],
       [0.        ]])

Again, the difference between the angular velocities of FTS and PyStokes correctly equals G2a2s * F2s appearing in the equation for the angular velocity. 

_____________________

Here are the matrix elements that have been used for FTS

<img src="./images/IMG_1347.jpeg" style="width: 500px;"/>

<img src="./images/IMG_1348.jpeg" style="width: 500px;"/>